In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import string
import re

In [2]:
df = pd.read_csv('imdb_master.csv',encoding="ISO-8859-1")

In [3]:
df = df.drop(['Unnamed: 0', 'type','file'], axis=1)
df = df[df.label != 'unsup']

In [4]:
df['label'] = df['label'].str.lower().replace({'pos': 1, 'neg': 0})
# df.tail(60)

,review,label
49940,Oh yeah! Jenna Jameson did it again! Yeah Baby...,1
49941,This one is quite a nice surprise. Cute little...,1
49942,I watched this movie once and might watch it a...,1
49943,Very nice action with an interwoven story whic...,1
49944,If you've ever wondered why they don't make po...,1
49945,Before watching this movie I thought this movi...,1
49946,this is really films outside (not in a motel r...,1
49947,This is a more interesting than usual porn mov...,1
49948,Saw this movie twice at community screenings a...,1
49949,Beforehand Notification: I'm sure someone is g...,1


In [5]:
import argparse
import time, random
from tqdm import tqdm
from torchtext import data
import numpy as np
import argparse
import os

In [6]:
df.review[1]

"This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying...<br /><br />Overall, this is second-rate action trash. There are countless better films to see, and if you really want to s

In [7]:
import string
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

exclude = set(string.punctuation)

def cleanup(text):
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in exclude)
    text = ''.join([i for i in text if not i.isdigit()])
    text = text.split()
    texts = []
    for word in text:
        word = lemmatizer.lemmatize(word)
        texts.append(word)
    return texts

[nltk_data] Downloading package wordnet to /Users/ribhu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
corpus = []
for sent in df.review:
    corpus.append(cleanup(sent))

In [9]:
labels = []
for label in df.label:
    labels.append(label)

In [10]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
wordvec = KeyedVectors.load_word2vec_format(filename, binary=True)

In [11]:
wordvec.get_vector('but')

array([-4.76074219e-02,  8.15429688e-02,  4.56542969e-02,  9.17968750e-02,
       -1.47094727e-02,  1.11328125e-01,  6.54296875e-02, -9.66796875e-02,
        1.38671875e-01,  1.43554688e-01, -7.86132812e-02, -1.63085938e-01,
       -9.17968750e-02, -7.12890625e-02, -8.05664062e-02,  8.83789062e-02,
        2.17773438e-01,  9.96093750e-02,  1.19018555e-02, -1.35742188e-01,
       -5.78613281e-02,  9.03320312e-03,  1.75781250e-01, -1.17187500e-02,
        4.90722656e-02, -1.05468750e-01, -1.53320312e-01,  6.17675781e-02,
        8.64257812e-02,  2.25830078e-02,  1.59740448e-05,  6.39648438e-02,
        2.33154297e-02, -1.45874023e-02,  1.51367188e-02,  5.29785156e-02,
       -1.34277344e-02,  7.99560547e-03,  2.63671875e-02,  2.99072266e-02,
        1.61132812e-01,  5.68847656e-02,  1.18164062e-01, -3.12500000e-02,
       -7.59887695e-03,  6.64062500e-02, -1.08032227e-02, -6.53076172e-03,
        1.13281250e-01,  9.22851562e-02,  4.29687500e-02, -6.10351562e-03,
        2.80761719e-02, -

In [12]:
#Splitting corpus of sentences into train, test, validation
sen_vec = []
X_train = []
X_cv = []
X_test = []
corp_train = corpus[0:200]
corp_dev = corpus[200:300]
corp_test = corpus[300:400]
for sentence in corp_train:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_train.append(input_vec)
    sen_vec = []
for sentence in corp_dev:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_cv.append(input_vec)
    sen_vec = []
for sentence in corp_test:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_test.append(input_vec)
    sen_vec = []
                                    #Should try with randoms as a variation if shit don't work



In [13]:
#SPlitt
labels_train = labels[0:200]
labels_cv = labels[200:300]
labels_test = labels[300:400]

In [33]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(wordvec.vectors))
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        y = self.hidden2tag(lstm_out[-1])
        log_probs = F.log_softmax(y,dim=1)
        return log_probs

In [34]:
# EMBEDDING_DIM = 300
# HIDDEN_DIM = 150
# EPOCHS = 20
# avg_loss = 0.0
# Hard_num = 200

# parameters = filter(lambda p: p.requires_grad, model.parameters())
# model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 300000, 2)
# loss_function = nn.NLLLoss()
# optimizer = optim.Adam(parameters,lr = 1e-3)
# for epoch in range(Hard_num):
#     model.zero_grad()
#     model.hidden = model.init_hidden()
#     sentence_in = inputs[epoch]
#     target = torch.tensor(labels[epoch], dtype=torch.long)
#     tag_scores = model(sentence_in)
#     loss = loss_function(tag_scores, target.unsqueeze_(0))
#     loss.backward()
#     optimizer.step()
# avg_loss /= Hard_num

In [35]:
def train():
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 150
    EPOCH = 20
    best_dev_acc = 0.0
    train_size = len(labels_train)
    dev_size = len(labels_cv)
    test_size= len(labels_test)
    model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 300000, 2)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    loss_function = nn.NLLLoss()
    optimizer = optim.Adam(parameters,lr = 1e-3)
    #optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
    no_up = 0
    for i in range(EPOCH):
        print('epoch: %d start!' % i)
        train_epoch(model, train_size, loss_function, optimizer, X_train, labels_train, i)
        print('now best dev acc:',best_dev_acc)
        dev_acc = evaluate(model,dev_size,loss_function,X_cv, labels_cv,'dev')
        test_acc = evaluate(model, test_size, loss_function, X_test, labels_test, 'test')
        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            os.system('rm mr_best_model_acc_*.model')
            print('New Best Dev!!!')
#             torch.save(model.state_dict(), 'best_models/mr_best_model_acc_' + str(int(test_acc*10000)) + '.model')
            no_up = 0
        else:
            no_up += 1
            if no_up >= 10:
                exit()

In [36]:
def get_accuracy(truth, pred):
    assert len(truth)==len(pred)
    right = 0
    for i in range(len(truth)):
        if truth[i]==pred[i]:
            right += 1.0
    return right/len(truth)

def train_epoch(model, datapoints, loss_function, optimizer, text, labels, i):
    model.train()
    
    avg_loss = 0.0
    count = 0
    truth_res = []
    pred_res = []
    batch_sent = []
    
    truth_res = labels

    for j in range(datapoints):
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        sent = text[j]
        label = torch.tensor(labels[j], dtype=torch.long)
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res.append(pred_label)
        model.zero_grad()
        loss = loss_function(pred, label.unsqueeze_(0))
        avg_loss += loss.data[0]
        count += 1
        if count % 500 == 0:
            print('epoch: %d iterations: %d loss :%g' % (i, count, loss.data[0]))

        loss.backward()
        optimizer.step()
    avg_loss /= datapoints
    print('epoch: %d done! \n train avg_loss:%g , acc:%g'%(i, avg_loss, get_accuracy(truth_res,pred_res)))

def evaluate(model, datapoints, loss_function, text, labels, name ='dev'):
    model.eval()
    avg_loss = 0.0
    truth_res = labels
    pred_res = []

    for j in range(datapoints):
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        sent = text[j]
        label = torch.tensor(labels[j], dtype=torch.long)
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res.append(pred_label)
        # model.zero_grad() # should I keep this when I am evaluating the model?
        loss = loss_function(pred, label.unsqueeze_(0))
        avg_loss += loss.data[0]
    avg_loss /= datapoints
    acc = get_accuracy(truth_res, pred_res)
    print(name + ' avg_loss:%g train acc:%g' % (avg_loss, acc ))
    return acc

In [37]:
train()

epoch: 0 start!


RuntimeError: Expected hidden[0] size (2, 1, 150), got (1, 1, 150)